<a href="https://colab.research.google.com/github/NILEG/TensorFlow/blob/main/NLP_002(Word_Embedding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

####Loading IMBD Data

In [2]:
imdb, info=tfds.load("imdb_reviews", as_supervised=True, with_info=True)
imdb

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEADR7M/imdb_reviews-train.tfrecord…

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEADR7M/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEADR7M/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [3]:
#Train and Test
train=imdb["train"]
test=imdb["test"]

In [4]:
#Making sentences and labels list
train_sentences=[]
train_labels=[]
test_sentences=[]
test_labels=[]
for sen, label in train:
  train_sentences.append(sen.numpy().decode("utf8"))
  train_labels.append(label.numpy())

for sen, label in test:
  test_sentences.append(sen.numpy().decode("utf8"))
  test_labels.append(label.numpy())


In [5]:
train_len=len(train_sentences)
test_len=len(test_sentences)
train_len, test_len

(25000, 25000)

In [6]:
train_labels[100]

1

####Creating Tokenizer

In [7]:
num_words=10000
max_length=120
dims=16
oov_token="<OOV>"
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

sequences_train=tokenizer.texts_to_sequences(train_sentences)
pad_sequence_train=tf.keras.preprocessing.sequence.pad_sequences(sequences_train, maxlen=max_length, truncating="post")

sequences_test=tokenizer.texts_to_sequences(test_sentences)
pad_sequence_test=tf.keras.preprocessing.sequence.pad_sequences(sequences_test, maxlen=max_length, truncating="post")



####Word Embedding Model

In [8]:
model=tf.keras.Sequential([tf.keras.layers.Input(shape=(120)),
                           tf.keras.layers.Embedding(num_words, dims),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=6, activation="relu"),
                           tf.keras.layers.Dense(units=1, activation="sigmoid")])
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.fit(pad_sequence_train, np.array(train_labels), epochs=10, validation_data=(pad_sequence_test, np.array(test_labels)))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.5221 - accuracy: 0.7334 - val_loss: 0.4024 - val_accuracy: 0.8146
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3085 - accuracy: 0.8696 - val_loss: 0.3755 - val_accuracy: 0.8322
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2256 - accuracy: 0.9124 - val_loss: 0.4057 - val_accuracy: 0.8256
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1472 - accuracy: 0.9490 - val_loss: 0.4626 - val_accuracy: 0.8126
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0793 - accuracy: 0.9763 - val_loss: 0.5688 - val_accuracy: 0.8020
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0338 - accuracy: 0.9921 - val_loss: 0.6904 - val_accuracy: 0.7942
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0125 - accuracy: 0.9970 - val_loss: 0.8416 - val_accuracy: 0.7872
Epoch 

####Visualizing

In [15]:
#Getting Weights
layers=model.layers
weights=layers[0].get_weights()[0]
weights.shape

(10000, 16)

In [ ]:
#Mapping Weights to each word using index to word dictionary

#=====word-to-index======
word_index=tokenizer.word_index #word as key
word_index

#======index-to-word=======
index_word=tokenizer.index_word
index_word

In [20]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8') #For Writing vector of each word
out_m = io.open('meta.tsv', 'w', encoding='utf-8') #For weights to each word

for i in range(1,num_words):
  word=index_word[i]
  vectors=weights[i]

  out_m.write(word+"\n")

  out_v.write("\t".join([str(x) for x in vectors])+"\n")

out_v.close()
out_m.close()

Upload these two files on projector.tensorflow.ord